In [1]:
import pandas as pd
import numpy as np

In [2]:
#read data
calories4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/calories.json')
distance4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/distance.json')
l_act4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/lightly_active_minutes.json')
m_act4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/moderately_active_minutes.json')
r_heart4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/resting_heart_rate.json')
sedentary4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/sedentary_minutes.json')
sleep_score4 = pd.read_csv('../osfstorage-archive/pmdata/p04/fitbit/sleep_score.csv')
sleep4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/sleep.json')
steps4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/steps.json')
hr_zone4 = pd.read_json('../osfstorage-archive/pmdata/p04/fitbit/time_in_heart_rate_zones.json')
v_act4 = pd.read_json('../osfstorage-archive/pmdata/p04//fitbit/very_active_minutes.json')

In [3]:
#data frequency to date using sum
calories_processed = calories4.resample('D', on = 'dateTime').value.sum()
distance_processed = distance4.resample('D', on = 'dateTime').value.sum()
steps_processed = steps4.resample('D', on = 'dateTime').value.sum()

In [4]:
#resting heart rate to simple df
r_heart4['value']=r_heart4['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart4

In [5]:
#delete time in sleep score time stamp
sleep_score4['timestamp']=sleep_score4['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score4.drop(columns='sleep_log_entry_id') #drop entry id

In [6]:
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone4['value']])
hr_zone_processed = pd.concat([hr_zone4['dateTime'], hr_zone_value], axis=1)

In [7]:
#unify index
l_act_processed = l_act4.set_index('dateTime')
m_act_processed = m_act4.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary4.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep4.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act4.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [8]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [9]:
#read prediction target
wellness = pd.read_csv('../osfstorage-archive/pmdata/p04/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [10]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [11]:
#concat data
P04 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [12]:
#rename columns
P04.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [13]:
P04

,readiness,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-01,5.0,1324.80,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-02,5.0,1324.80,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-03,5.0,1324.80,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-04,5.0,1324.80,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-05,6.0,1648.80,NaN,0,0,NaN,1390,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,6.0,1855.88,480060.0,185,14,67.089778,679,89.0,23.0,20.0,...,0.0,544.0,93.0,1.0,7149.0,0.0,0.0,73.0,1366.0,18
2020-03-28,5.0,2172.80,876790.0,141,65,66.752407,572,88.0,23.0,20.0,...,0.0,614.0,93.0,1.0,13075.0,5.0,0.0,182.0,1040.0,48
2020-03-29,4.0,1891.53,535410.0,149,26,67.571217,754,84.0,22.0,19.0,...,0.0,548.0,95.0,1.0,7970.0,0.0,0.0,85.0,1289.0,23
2020-03-30,6.0,1997.46,618770.0,173,15,66.596192,559,90.0,22.0,22.0,...,6.0,647.0,96.0,1.0,9210.0,8.0,0.0,104.0,1322.0,46
